# VaR System: Simulated Dataset and Design Spec
In this notebook we will create a simulated dataset of a trading firm's positions and their associated simulated historical PnL for use in computing VaR of individual portfolios and custom groupings. We'll then spec out a system to compute VaR on arbitrary sub-aggregations.

## Disclaimer
This notebook contains no MLP proprietary data and does not depend on any external data sources inside or outside MLP. All data is purely hypothetical, generated from random distributions to simulate the broad scale and shape of the dataset we are working with. It is intended for use in an academic software engineering and design project and should not be interpreted as having any similarity to MLP's actual position and PnL data at the current time or at any time in the past.

In [1]:
import numpy as np
import pandas as pd
import random
import string
import csv

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from scipy.stats import t

In [2]:
# Rough scale of the data set we are working with
n_securities = 200000
n_positions = 2000000
n_desks = 20
n_pods = 200
n_traders = 2000
n_micros = 20000

## 1. Generate a Security Universe
First order of business is to create a universe of tradable securities that our investment professionals can take positions in. We will create 200,000 hypothetical securities.

In [3]:
def generate_samples(n_samples, distinct_strings):
    """
    Generates the desired number of samples from a provided list of distinct strings, such that the result is approximately
    Gaussian, and the frequencies of the samples are not uniform. This is a better approximation of real datasets than an even
    random sample, and will allow us to more easily test the performance of the system for queries that have heterogeneous
    result set sizes.
    """
    # Create a probability distribution that approximates a Gaussian distribution
    n_distinct = len(distinct_strings)
    mean = n_distinct / 2
    std_dev = n_distinct / 4
    x = np.linspace(0, n_distinct - 1, n_distinct)
    weights = np.exp(-((x - mean) ** 2) / (2 * std_dev ** 2))
    weights /= np.sum(weights)  # Normalize the weights so they sum up to 1

    return np.random.choice(distinct_strings, size=n_samples, p=weights)

def generate_random_strings(n, length):
    """
    Generates the desired number of strings of the specified length, constructed from random alphanumeric characters. 
    The list is guaranteed to contain distinct values, no repeats.
    """
    base = string.ascii_uppercase + string.digits
    random_strings = set()
    
    while len(random_strings) < n:
        rnd_string = ''.join(random.choice(base) for _ in range(length))
        random_strings.add(rnd_string)
        
    return list(random_strings)
    

In [4]:
security_id_space = generate_random_strings(n_securities, 8)
print(security_id_space[:5])

['IWWSMODR', 'MQ4D9Z22', 'OXKH7BOH', '0PX4N5MM', 'T5RQZIZS']


In [17]:
security_universe = pd.DataFrame({
    "securityid": security_id_space,
    "assetclass": generate_samples(n_securities, ["Equity", "FixedIncome", "Mortgage", "Commodity", "Digital"]),
    "securitytype": generate_samples(n_securities, ["Stock", "Bond", "Future", "Option", "ETF", "Swap", "CDS", "Index"]),
    "tradingcountry": generate_samples(n_securities, ["US", "GB", "JP", "FR", "ES", "IT", "AU", "GR", "BR", "IN", "CN", "CH", "CA"]),
    "tradingcurrency": generate_samples(n_securities, ["USD", "GBP", "JPY", "EUR", "INR", "CNY", "CHF", "CAD", "AUD"]),
    "issuername": generate_samples(n_securities, generate_random_strings(1757, 8)),
    "ticker": generate_samples(n_securities, generate_random_strings(1393, 8)),
    "rating": generate_samples(n_securities, ["AAA", "AA", "A", "BBB", "BB", "B", "CCC", "CC", "C"]),
    "industrygroup": generate_samples(n_securities, generate_random_strings(25, 8)),
    "industry": generate_samples(n_securities, generate_random_strings(74, 8)),
    "securityname": generate_samples(n_securities, generate_random_strings(920, 8)),
    "maturitydate": generate_samples(n_securities, generate_random_strings(1368, 8)),
    "underlyingname": generate_samples(n_securities, generate_random_strings(949, 8)),
    "regionname": generate_samples(n_securities, ["Americas", "EMEA", "Asia"]),
    "issuercountryofrisk": generate_samples(n_securities, generate_random_strings(80, 8))
})

security_universe.head()

KeyError: 0

In [10]:
# Check the distribution shape
print(security_universe['securitytype'].value_counts())

securitytype
ETF       41952
Option    36980
Swap      36868
Future    25621
CDS       25236
Index     13911
Bond      13724
Stock      5708
Name: count, dtype: int64


In [32]:
# Generate a csv for securities 
with open('./data/securities.csv', 'w', newline='') as csvfile:
    # columns for the securities dataframe (security_universe.columns)
    securities_cols = ['securityid', 'assetclass', 'securitytype', 'tradingcountry',
       'tradingcurrency', 'issuername', 'ticker', 'rating', 'industrygroup',
       'industry', 'securityname', 'maturitydate', 'underlyingname',
       'regionname', 'issuercountryofrisk']
    
    writer = csv.writer(csvfile)
    writer.writerow(securities_cols)
    
    added = 0
    for i in security_universe.index:
        row = []
        for col in securities_cols: 
            row.append(security_universe[col][i])
        writer.writerow(row)

## 2. Generate a Position Snapshot
Now we need a dataset that simulates the holdings of the firm at any given point in time. This data will identify each trader's  holdings by security ID, with a `quantity` (# of units of the security) associated with each holding.
### Disclaimer
This is a purely random set of hypothetical positions and has no relationship to MLP's actual positions at present or at any point in the past.

In [25]:
def generate_random_position_quantities(n_samples, mean=0, std_dev=100):
    """
    Generates a random set of quantities to use for the positions, using a Gaussian approach but set to output
    fairly round numbers.
    """
    # Generate extra samples to account for the ones we'll remove
    samples = np.random.normal(mean, std_dev / 10, size=int(n_samples*1.4))
    
    # Round to nearest whole number of tens
    samples = np.round(samples).astype(int) * 10
    
    # Remove zeros, a zero position is not an active position and should be ignored
    samples = samples[samples != 0]
    
    # Trim the array to the desired size
    samples = samples[:n_samples]
    
    return samples

quantities = generate_random_position_quantities(n_samples=n_positions)
print(quantities[:5])

[ 200   80 -170  -90  120]


In [26]:
def generate_samples_bucketed(n_samples, distinct_strings, parent):
    """
    Generates the desired number of samples from a provided list of distinct strings, such that the frequencies of the 
    samples are Gaussian and not uniform, AND such that each occurrence of a given string in the output array occurs in
    a position containing the same value in the input "parent" array.

    Args:
        n_samples (int): The number of samples to generate.
        distinct_strings (list): The list of distinct strings to sample from.
        parent (list): The list of parent values. Each unique value in this list corresponds to a distinct subset of 
                       distinct_strings.

    Returns:
        list: The generated samples.
    """
    # Create a probability distribution that approximates a Gaussian distribution
    n_distinct = len(distinct_strings)
    mean = n_distinct / 2
    std_dev = n_distinct / 4  # Arbitrary choice, you can adjust this
    x = np.linspace(0, n_distinct - 1, n_distinct)
    weights = np.exp(-((x - mean) ** 2) / (2 * std_dev ** 2))
    weights /= np.sum(weights)  # Normalize the weights so they sum up to 1

    # Create a mapping from each unique parent value to a distinct subset of distinct_strings
    unique_parents = np.unique(parent)
    n_unique = len(unique_parents)
    subset_size = n_distinct // n_unique
    subsets = {p: distinct_strings[i*subset_size:(i+1)*subset_size] for i, p in enumerate(unique_parents)}
    subset_weights = {p: weights[i*subset_size:(i+1)*subset_size] for i, p in enumerate(unique_parents)}
    for p in unique_parents:
        subset_weights[p] /= subset_weights[p].sum()  # Normalize the weights for each subset

    # Generate the samples
    samples = [np.random.choice(subsets[parent[i]], p=subset_weights[parent[i]]) for i in range(n_samples)]

    return samples

In [27]:
supervisors = generate_samples(n_positions, ["Fundamental EQ", "Fixed Income", "Quant Arb", "Commod", "Other"])

desks = generate_samples_bucketed(n_positions, generate_random_strings(n_desks, 8), parent=supervisors)
print("Done generating desks")

pods = generate_samples_bucketed(n_positions, generate_random_strings(n_pods, 8), parent=desks)
print("Done generating pods")

traders = generate_samples_bucketed(n_positions, generate_random_strings(n_traders, 8), parent=pods)
print("Done generating traders")

microstrategies = generate_samples_bucketed(n_positions, generate_random_strings(n_micros, 8), parent=traders)
print("Done generating micros")

positions = pd.DataFrame({
    "securityid": generate_samples(n_positions, security_id_space),
    "trader": traders,
    "microstrategy": microstrategies,
    "pod": pods,
    "desk": desks,
    "supervisor": supervisors,
    "quantity": quantities
})


Done generating desks
Done generating pods
Done generating traders
Done generating micros


,securityid,trader,microstrategy,pod,desk,supervisor,quantity
0,YE8AV40Z,1XS3KOD8,PZSIKZM2,XA1V5EH9,5R6QR2L9,Commod,200
1,89HNLZDP,J3C2A6II,SCOS81Y2,D3SFZXBG,I22Q0W84,Commod,80
2,PHJY55AG,HB6RIBED,QIEAKT7U,BPSKBLAX,Q2TZVQJK,Fixed Income,-170
3,5W4LJGN4,L19RYQ3A,FWW4RDDH,XCFX9L3D,9LZBU75X,Quant Arb,-90
4,RPRP5ZXO,F8L7ZCMM,2FRVB1RJ,Z7EWQ5DK,JFQRCCHV,Other,120


In [30]:


positions.head()
positions.shape # 2,000,000 x 7 
positions.columns

Index(['securityid', 'trader', 'microstrategy', 'pod', 'desk', 'supervisor',
       'quantity'],
      dtype='object')

In [33]:
# Check the distribution shape
print(positions['pod'].value_counts()[:3])
print(positions['pod'].value_counts()[99:102])
print(positions['pod'].value_counts()[-3:])

pod
791JLCAB    28058
1RGMSJGH    26904
D0H4U6N5    26239
Name: count, dtype: int64
pod
EOC1R967    9044
I0WY3H37    8892
9W16Q6G1    8854
Name: count, dtype: int64
pod
LSBGJQGO    2031
JA3RTK5G    2024
68E1V89Y    2010
Name: count, dtype: int64


In [34]:
# Check fidelity of hierarchy assignments
# All occurrences of a given microstrategy should fall under the same trader, pod, desk and supervisor

smallest_microstrategy = positions['microstrategy'].value_counts()[-1:].index[-1]
positions[positions['microstrategy'] == smallest_microstrategy]

,securityid,trader,microstrategy,pod,desk,supervisor,quantity
250055,5KZAHWDR,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-60
262282,Z56Z3HZP,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-50
548251,Y4UGZEIL,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,10
592355,0KOFAEP8,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,100
821427,AATRDG7Q,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-80
838907,TBFO7K14,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,40
1061317,0HSM0M76,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-50
1345565,PR0ANKG0,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-70
1461110,7P2LXLAA,V35P78QE,XU9L4UF4,LSBGJQGO,KNTAIU34,Fundamental EQ,-10


In [35]:
# Generate a csv for positions 
with open('./data/positions.csv', 'w', newline='') as csvfile:
    # columns for the securities dataframe (security_universe.columns)
    position_cols = ['securityid', 'trader', 'microstrategy', 'pod', 'desk', 'supervisor', 'quantity']
    
    writer = csv.writer(csvfile)
    writer.writerow(position_cols)
    
    added = 0
    for i in positions.index:
        row = []
        for col in position_cols: 
            row.append(positions[col][i])
        writer.writerow(row)

## 3. Generate the historical simulated PnL for each security 
The backbone of our VaR calculations is a dataset consisting of a set of simulated per-unit price changes (P&L) in response to all daily market moves observed during the last ~15 years, for each of the 200K securities in our tradable universe. 

A single row in this dataset may be interpreted as follows:
```
securityid   date        pnl
WDANCZU5     2024-01-03  -381.4
```
--> "If the same relative market movements that happened between close of business 2024-01-02 and close of business 2024-01-03 were to happen between yesterday's market close and today's market close, the market price of a single unit of security WDANCZU5 will decrease by $381.40."

Here, the data volume gets too large to keep in dataframes (5,000 dates * 200,000 securities = 1,000,000,000 total rows), so we'll write out to the disk. Make sure you have plenty of hard drive space before running this step.

### Disclaimer

Although we will use the term "PnL" to describe this dataset, it is purely random and hypothetical, and has no relationship to the PnL ever experienced by any security or position held by MLP or otherwise, now or at any time in the past.

In [37]:
# Generate the historical simulation dates: all weekdays since 2008

start_date = datetime(2008, 1, 1)
end_date = datetime.now()
weekdays = []

for n in range(int ((end_date - start_date).days)):
    current_date = start_date + timedelta(n)
    # Check if current day is a weekday (0: Monday, 1: Tuesday, ..., 4: Friday)
    if current_date.weekday() < 5:
        # Append the date in "YYYY-MM-DD" format
        weekdays.append(current_date.strftime('%Y-%m-%d'))

print(weekdays[:8])

['2008-01-01', '2008-01-02', '2008-01-03', '2008-01-04', '2008-01-07', '2008-01-08', '2008-01-09', '2008-01-10']


In [13]:
# We're going to use a Student's T with degrees of freedom = 3 to generate the simulated PnL. We don't want to use
# a Gaussian for this as real life PnL is not Gaussian (if it were, risk management for hedge funds would be much simpler).

degrees_of_freedom = 3  # fat tail
t_distribution = t(3)
standard_deviation = 100

In [36]:
with open('simulated_pnl.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['securityid', 'date', 'pnl'])
    
    debug_lines = 0
    for security_id in security_id_space:
        for weekday in weekdays:
            row = [security_id, weekday, round(t_distribution.rvs() * standard_deviation, 2)]
            writer.writerow(row)
            
            if debug_lines < 10:
                print(row)
            
            debug_lines += 1

NameError: name 'weekdays' is not defined

## 4. Spec out the queries that the system will need to support
We want to build a system that can serve up percentiles of the above data, applying filters to one or more of the position or security columns to select a sub-population, aggregating to one or more of the columns, and then computing a percentile at each level of the aggregation. For example:
```
select
    p.desk, p.pod, s.assetclass, percentile(0.05, x.pnl)
from
    position p, security s, simulated_pnl x
where
    p.securityid = s.securityid and x.securityid = s.securityid
    and p.supervisor = 'Fundamental EQ'
    and s.tradingcountry = 'US'
group by
    p.desk, p.pod, p.assetclass
```
Note that your implementation **does not** need to actually take SQL as input! You are free to define the API as you choose, it could be JSON, GraphQL, etc.

### Functional and Performance Requirements

1. Given a multi-threaded workload consisting of heterogeneous queries like the above, the system should be able to maintain a **1 qps (query per second) throughput under load**. You will need to build a test harness to generate heterogeneous queries with different filters and aggregation dimensions in your chosen API input format.
1. The system is allowed to impose a hard limit of 100K output rows per query. Very large aggregations (e.g. those resulting from a "group by" on multiple high-cardinality dimensions) are not a typical use case and the response payload may be too large for the client to handle anyway. The system may return a 400 Bad Request error if it detects that the output will be >100K rows.
1. The typical percentiles users will need are 0.01, 0.05, 0.95, 0.99, min and max. If your API design and architecture support free-form percentiles, then fine, but if not, you are free to support only these values as a fixed enum.


